# Fuction calling

In [1]:
import { load } from "dotenv";
const env = await load();

In [8]:
import OpenAI from "openai";

const openai = new OpenAI({
    apiKey: env["AZURE_OPENAI_API_KEY"],
    baseURL: `https://${env["AZURE_OPENAI_API_INSTANCE_NAME"]}.openai.azure.com/openai/deployments/${env["AZURE_OPENAI_API_DEPLOYMENT_NAME"]}`,
    defaultQuery: { 'api-version':  env["AZURE_OPENAI_API_VERSION"] },
    defaultHeaders: { 'api-key': env["AZURE_OPENAI_API_KEY"] },
});

In [2]:
function getCurrentWeather({ location, unit="fahrenheit"}){
    const  weather_info = {
         "location": location,
         "temperature": "72",
         "unit": unit,
         "forecast": ["sunny", "windy"],
     }
     return JSON.stringify(weather_info);
 }

In [4]:

/**
 * @param {string} location - The city and state, e.g. San Francisco, CA
 * @param {string} unit - The unit of temperature, e.g. celsius or fahrenheit
 * 
 * @returns {object} The current weather in the given location
 * 
 */

const tools = [
    {
      type: "function",
      function: {
        name: "getCurrentWeather",
        description: "Get the current weather in a given location",
        parameters: {
          type: "object",
          properties: {
            location: {
              type: "string",
              description: "The city and state, e.g. San Francisco, CA",
            },
            unit: { type: "string", enum: ["celsius", "fahrenheit"] },
          },
          required: ["location"],
        },
      },
    }
]

In [15]:
const messages = [
    {
        "role": "user",
        "content": "北京的天气怎么样"
    }
]

const result = await openai.chat.completions.create({
    model: 'gpt-3.5-turbo',
    messages,
    tools
  });
  console.log(result.choices[0]);

{
  content_filter_results: {},
  finish_reason: "tool_calls",
  index: 0,
  message: {
    content: null,
    role: "assistant",
    tool_calls: [
      {
        function: {
          arguments: '{\n  "location": "北京"\n}',
          name: "getCurrentWeather"
        },
        id: "call_PKni9WAnjT3a6h8tWZVUOfHB",
        type: "function"
      }
    ]
  }
}


In [14]:
// const messages = [
//     {
//         "role": "user",
//         // "content": "北京的天气怎么样"
//         "content": "What's the weather like in Beijing?"

//     }
// ]

// const result = await openai.chat.completions.create({
//     // model: 'gpt-3.5-turbo',
//     messages,
//     tools
//   });


//   console.log(result.choices[0]);

In [16]:
const messages = [
    {
        "role": "user",
        "content": "北京的天气怎么样"
    }
]

const result = await openai.chat.completions.create({
    model: 'gpt-3.5-turbo',
    messages,
    tools,
    tool_choice: "none"
  });
  console.log(result.choices[0]);

{
  content_filter_results: {
    hate: { filtered: false, severity: "safe" },
    self_harm: { filtered: false, severity: "safe" },
    sexual: { filtered: false, severity: "safe" },
    violence: { filtered: false, severity: "safe" }
  },
  finish_reason: "stop",
  index: 0,
  message: { content: "请稍等，我来查一下北京的天气。", role: "assistant" }
}


In [17]:
const messages = [
    {
        "role": "user",
        "content": "你好"
    }
]

const result = await openai.chat.completions.create({
    model: 'gpt-3.5-turbo',
    messages,
    tools,
    tool_choice: {
        type: "function",
        function: {
           name: "getCurrentWeather"
        }
    }
  });
 
 console.log(result.choices[0]);

{
  content_filter_results: {},
  finish_reason: "stop",
  index: 0,
  message: {
    content: null,
    role: "assistant",
    tool_calls: [
      {
        function: {
          arguments: '{\n  "location": "北京"\n}',
          name: "getCurrentWeather"
        },
        id: "call_p6he7JQ2h1XDHnI4lBhhDT1S",
        type: "function"
      }
    ]
  }
}


In [18]:
const functions = {
    "getCurrentWeather": getCurrentWeather
  }

const functionInfo = result.choices[0].message.tool_calls[0].function
const functionName = functionInfo.name;
const functionParams = functionInfo.arguments

const functionResult = functions[functionName](functionParams);

console.log(functionResult);

{"temperature":"72","unit":"fahrenheit","forecast":["sunny","windy"]}


## Concurrence of two functions

In [19]:
function getCurrentTime({ format = "iso" } = {}) {
    let currentTime;
    switch (format) {
        case "iso":
            currentTime = new Date().toISOString();
            break;
        case "locale":
            currentTime = new Date().toLocaleString();
            break;
        default:
            currentTime = new Date().toString();
            break;
    }
    return currentTime;
}


In [27]:
const tools = [
  {
      type: "function",
      function: {
          name: "getCurrentTime",
          description: "Get the current time in a given format",
          parameters: {
              type: "object",
              properties: {
                  format: {
                      type: "string",
                      enum: ["iso", "locale", "string"],
                      description: "The format of the time, e.g. iso, locale, string",
                  },
              },
              required: ["format"],
          },
      },
  },
  {
      type: "function",
      function: {
        name: "getCurrentWeather",
        description: "Get the current weather in a given location",
        parameters: {
          type: "object",
          properties: {
            location: {
              type: "string",
              description: "The city and state, e.g. San Francisco, CA",
            },
            unit: { type: "string", enum: ["celsius", "fahrenheit"] },
          },
          required: ["location", "unit"],
        },
      },
  }
]

const messages = [
    {
        "role": "user",
        "content": "请同时告诉我当前的时间, 和北京的天气"
    }
]

const result = await openai.chat.completions.create({
     model: 'gpt-3.5-turbo',
    messages,
    tools,
  });


console.log(result.choices[0]);

{
  content_filter_results: {},
  finish_reason: "tool_calls",
  index: 0,
  message: {
    content: null,
    role: "assistant",
    tool_calls: [
      {
        function: {
          arguments: '{\n  "format": "locale"\n}',
          name: "getCurrentTime"
        },
        id: "call_EPmyOutP3CKzrNjY3gC2GST2",
        type: "function"
      }
    ]
  }
}


In [28]:
const messages = [
    { role: "user", content: "北京天气如何?" },
]

const result = await openai.chat.completions.create({
    model: 'gpt-3.5-turbo',
    messages,
    tools
  });


In [29]:
messages.push(result.choices[0].message)

const functions = {
    "getCurrentWeather": getCurrentWeather
  }

const cell = result.choices[0].message.tool_calls[0]
const functionInfo = cell.function
const functionName = functionInfo.name;
const functionParams = functionInfo.arguments
const functionResult = functions[functionName](functionParams);

messages.push({
  tool_call_id: cell.id,
  role: "tool",
  name: functionName,
  content: functionResult,
}); 

3

In [30]:
console.log(messages);

[
  { role: "user", content: "北京天气如何?" },
  {
    content: null,
    role: "assistant",
    tool_calls: [
      {
        function: {
          arguments: '{\n"location": "北京",\n"unit": "celsius"\n}',
          name: "getCurrentWeather"
        },
        id: "call_75NK4bW5RsrTWyH74HdQdIMS",
        type: "function"
      }
    ]
  },
  {
    tool_call_id: "call_75NK4bW5RsrTWyH74HdQdIMS",
    role: "tool",
    name: "getCurrentWeather",
    content: '{"temperature":"72","unit":"fahrenheit","forecast":["sunny","windy"]}'
  }
]


In [31]:
const response = await openai.chat.completions.create({
    model: 'gpt-3.5-turbo',
    messages,
  });
  console.log(response.choices[0].message);

{ content: "北京的天气目前是晴天，温度为72华氏度，有风。", role: "assistant" }
